In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [2]:
rental = pd.read_csv('resources/data/Apartment-List-Rent-Data-State_2018-6.csv')
rental.head()

,Location,Location_Type,Bedroom_Size,Price_2014_01,Price_2014_02,Price_2014_03,Price_2014_04,Price_2014_05,Price_2014_06,Price_2014_07,...,Price_2017_09,Price_2017_10,Price_2017_11,Price_2017_12,Price_2018_01,Price_2018_02,Price_2018_03,Price_2018_04,Price_2018_05,Price_2018_06
0,Alabama,State,Studio,573,573,574,574,575,576,578.0,...,602,602,602,601,600,597,598,600,604,608
1,Alabama,State,1br,624,624,625,625,626,627,629.0,...,656,655,655,655,653,650,650,653,658,662
2,Alabama,State,2br,758,758,759,759,760,762,765.0,...,797,795,796,795,794,790,790,793,799,804
3,Alabama,State,3br,1013,1013,1014,1015,1016,1018,1022.0,...,1065,1063,1063,1063,1061,1055,1056,1060,1068,1074
4,Alabama,State,4br,1178,1178,1179,1180,1182,1184,1188.0,...,1238,1236,1237,1236,1234,1228,1228,1233,1242,1250


## Data Wrangling

### Assess 

In [3]:
rental.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 255 entries, 0 to 254
Data columns (total 57 columns):
Location         255 non-null object
Location_Type    255 non-null object
Bedroom_Size     255 non-null object
Price_2014_01    255 non-null int64
Price_2014_02    255 non-null int64
Price_2014_03    255 non-null int64
Price_2014_04    255 non-null int64
Price_2014_05    255 non-null int64
Price_2014_06    255 non-null int64
Price_2014_07    250 non-null float64
Price_2014_08    250 non-null float64
Price_2014_09    250 non-null float64
Price_2014_10    250 non-null float64
Price_2014_11    250 non-null float64
Price_2014_12    250 non-null float64
Price_2015_01    250 non-null float64
Price_2015_02    250 non-null float64
Price_2015_03    250 non-null float64
Price_2015_04    250 non-null float64
Price_2015_05    250 non-null float64
Price_2015_06    250 non-null float64
Price_2015_07    250 non-null float64
Price_2015_08    255 non-null int64
Price_2015_09    255 non-null int64
Pr

#### Findings
 - Location_type is a redundant column
 - Year and month are wide data instead of long
 - Only data to the end of 2017 is needed
 - Data is unavailable for pre-2014 (only four year)

### Clean

In [22]:
rental_copy = rental.copy()

Drop redundant columns 

In [23]:
rental_copy.drop(['Location_Type', 
                  'Price_2018_01', 
                  'Price_2018_02', 
                  'Price_2018_03', 
                  'Price_2018_04', 
                  'Price_2018_05', 
                  'Price_2018_06'], 
                 axis=1, inplace=True)
rental_copy.head()

,Location,Bedroom_Size,Price_2014_01,Price_2014_02,Price_2014_03,Price_2014_04,Price_2014_05,Price_2014_06,Price_2014_07,Price_2014_08,...,Price_2017_03,Price_2017_04,Price_2017_05,Price_2017_06,Price_2017_07,Price_2017_08,Price_2017_09,Price_2017_10,Price_2017_11,Price_2017_12
0,Alabama,Studio,573,573,574,574,575,576,578.0,580.0,...,595,597,599,601,603,603,602,602,602,601
1,Alabama,1br,624,624,625,625,626,627,629.0,631.0,...,647,650,652,655,656,656,656,655,655,655
2,Alabama,2br,758,758,759,759,760,762,765.0,766.0,...,787,789,792,795,797,797,797,795,796,795
3,Alabama,3br,1013,1013,1014,1015,1016,1018,1022.0,1024.0,...,1051,1055,1058,1063,1065,1065,1065,1063,1063,1063
4,Alabama,4br,1178,1178,1179,1180,1182,1184,1188.0,1191.0,...,1222,1227,1231,1236,1239,1239,1238,1236,1237,1236


Use pd.wide_to_long to transform month info to long

In [24]:
rental_copy = pd.wide_to_long(rental_copy, 
                              stubnames=['Price_2014', 'Price_2015', 'Price_2016', 'Price_2017'],
                              i=['Location', 'Bedroom_Size'],
                              j='month', 
                              sep='_').reset_index()

In [25]:
rental_copy.head()

,Location,Bedroom_Size,month,Price_2014,Price_2015,Price_2016,Price_2017
0,Alabama,Studio,01,573.0,580.0,580.0,592
1,Alabama,Studio,02,573.0,580.0,578.0,594
2,Alabama,Studio,03,574.0,580.0,576.0,595
3,Alabama,Studio,04,574.0,581.0,580.0,597
4,Alabama,Studio,05,575.0,582.0,583.0,599


In [26]:
rental_copy.shape

(3060, 7)

The number of rows should have multiplied by 12. This is correct. 

In [28]:
# Double check for duplicates
rental_copy.duplicated().sum()

0

Use pd.wide_to_long to transform year info to long

In [29]:
rental_copy = pd.wide_to_long(rental_copy, stubnames='Price', i=['Location', 'Bedroom_Size', 'month'], j='year', sep='_')\
    .reset_index()

In [30]:
rental_copy.head()

,Location,Bedroom_Size,month,year,Price
0,Alabama,Studio,01,2014,573.0
1,Alabama,Studio,01,2015,580.0
2,Alabama,Studio,01,2016,580.0
3,Alabama,Studio,01,2017,592.0
4,Alabama,Studio,02,2014,573.0


In [31]:
rental_copy.shape

(12240, 5)

In [33]:
rental_copy.iloc[:, :-1].duplicated().sum()

0

Change column headers to lower case

In [47]:
cols = ['location', 'bedroom_size', 'month', 'year', 'price']
rental_copy.columns = cols

In [11]:
rental_copy.columns

Index(['location', 'bedroom_size', 'month', 'year', 'price'], dtype='object')

Check data type of new columns

In [12]:
rental_copy.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12240 entries, 0 to 12239
Data columns (total 5 columns):
location        12240 non-null object
bedroom_size    12240 non-null object
month           12240 non-null object
year            12240 non-null object
price           12155 non-null float64
dtypes: float64(1), object(4)
memory usage: 478.2+ KB


Because month and year won't be used for calculations, ok with leaving them as is.

In [13]:
rental_copy.to_csv('datasets/rent_by_type_1417.csv', index=False)

### Create groupings by year and single and family housing types

Assumptions: 
- A single person will typically need a studio or one bedroom
- While it is possible that a couple is a family, it is assumed that families will typically need two or more bedrooms

In [52]:
df = pd.read_csv('datasets/rent_by_type_1417.csv')
df.head()

,location,bedroom_size,month,year,price
0,Alabama,Studio,1,2014,573.0
1,Alabama,Studio,1,2015,580.0
2,Alabama,Studio,1,2016,580.0
3,Alabama,Studio,1,2017,592.0
4,Alabama,Studio,2,2014,573.0


In [53]:
df.bedroom_size.unique()

array(['Studio', '1br', '2br', '3br', '4br'], dtype=object)

In [61]:
single = df[(df.bedroom_size == 'Studio') | (df.bedroom_size == '1br')]
single.head()

,location,bedroom_size,month,year,price
0,Alabama,Studio,1,2014,573.0
1,Alabama,Studio,1,2015,580.0
2,Alabama,Studio,1,2016,580.0
3,Alabama,Studio,1,2017,592.0
4,Alabama,Studio,2,2014,573.0


In [64]:
single = single.groupby(['location', 'year'])['price'].mean().reset_index()
single.head()

,location,year,price
0,Alabama,2014,602.458333
1,Alabama,2015,605.875000
2,Alabama,2016,610.833333
3,Alabama,2017,625.791667
4,Alaska,2014,912.750000


In [65]:
single.to_csv('datasets/single-rent-prices.csv', index=False)

In [62]:
family = df[(df.bedroom_size != 'Studio') & (df.bedroom_size != '1br')]
family.head()

,location,bedroom_size,month,year,price
96,Alabama,2br,1,2014,758.0
97,Alabama,2br,1,2015,767.0
98,Alabama,2br,1,2016,767.0
99,Alabama,2br,1,2017,783.0
100,Alabama,2br,2,2014,758.0


In [66]:
family = family.groupby(['location', 'year'])['price'].mean().reset_index()
family.head()

,location,year,price
0,Alabama,2014,989.333333
1,Alabama,2015,995.055556
2,Alabama,2016,1003.083333
3,Alabama,2017,1027.611111
4,Alaska,2014,1705.861111


In [67]:
family.to_csv('datasets/family-rent-prices.csv', index=False)

### Join rental data to tableau data

Tableau is not letting me save joined data so just joining in the notebook

In [68]:
tableau = pd.read_csv('datasets/tableau-data.csv')
tableau.head()

,year,total,individual,family,youth,child,yadult,yparent,yparent_u18,yparent_18to24,ypchild,state,population,per_capita
0,2017,1845.0,1354.0,491.0,162.0,15.0,147.0,22.0,0.0,22.0,39.0,Alaska,739795.0,2.493934
1,2017,3793.0,2985.0,808.0,294.0,36.0,258.0,29.0,6.0,23.0,39.0,Alabama,4874747.0,0.778092
2,2017,2467.0,2068.0,399.0,208.0,17.0,191.0,10.0,0.0,10.0,13.0,Arkansas,3004279.0,0.821162
3,2017,8947.0,6488.0,2459.0,578.0,55.0,523.0,81.0,0.0,81.0,112.0,Arizona,7016270.0,1.275179
4,2017,134278.0,112756.0,21522.0,15458.0,1649.0,13809.0,890.0,16.0,874.0,1058.0,California,39536653.0,3.396292


Join 'single' rental information

In [69]:
df = pd.merge(left=tableau, right=single, how='left', left_on=['state', 'year'], right_on=['location', 'year'])
df.head()

,year,total,individual,family,youth,child,yadult,yparent,yparent_u18,yparent_18to24,ypchild,state,population,per_capita,location,price
0,2017,1845.0,1354.0,491.0,162.0,15.0,147.0,22.0,0.0,22.0,39.0,Alaska,739795.0,2.493934,Alaska,878.625000
1,2017,3793.0,2985.0,808.0,294.0,36.0,258.0,29.0,6.0,23.0,39.0,Alabama,4874747.0,0.778092,Alabama,625.791667
2,2017,2467.0,2068.0,399.0,208.0,17.0,191.0,10.0,0.0,10.0,13.0,Arkansas,3004279.0,0.821162,Arkansas,544.416667
3,2017,8947.0,6488.0,2459.0,578.0,55.0,523.0,81.0,0.0,81.0,112.0,Arizona,7016270.0,1.275179,Arizona,744.416667
4,2017,134278.0,112756.0,21522.0,15458.0,1649.0,13809.0,890.0,16.0,874.0,1058.0,California,39536653.0,3.396292,California,1308.875000


Drop location

In [70]:
df.drop('location', axis=1, inplace=True)
df.columns

Index(['year', 'total', 'individual', 'family', 'youth', 'child', 'yadult',
       'yparent', 'yparent_u18', 'yparent_18to24', 'ypchild', 'state',
       'population', 'per_capita', 'price'],
      dtype='object')

Change `price` to `individual rent`

In [72]:
df.rename(columns={'price': 'individual_rent'}, inplace=True)
df.columns

Index(['year', 'total', 'individual', 'family', 'youth', 'child', 'yadult',
       'yparent', 'yparent_u18', 'yparent_18to24', 'ypchild', 'state',
       'population', 'per_capita', 'individual_rent'],
      dtype='object')

Join 'family' rental information

In [73]:
df = pd.merge(left=df, right=family, how='left', left_on=['state', 'year'], right_on=['location', 'year'])
df.head()

,year,total,individual,family,youth,child,yadult,yparent,yparent_u18,yparent_18to24,ypchild,state,population,per_capita,individual_rent,location,price
0,2017,1845.0,1354.0,491.0,162.0,15.0,147.0,22.0,0.0,22.0,39.0,Alaska,739795.0,2.493934,878.625000,Alaska,1642.000000
1,2017,3793.0,2985.0,808.0,294.0,36.0,258.0,29.0,6.0,23.0,39.0,Alabama,4874747.0,0.778092,625.791667,Alabama,1027.611111
2,2017,2467.0,2068.0,399.0,208.0,17.0,191.0,10.0,0.0,10.0,13.0,Arkansas,3004279.0,0.821162,544.416667,Arkansas,942.305556
3,2017,8947.0,6488.0,2459.0,578.0,55.0,523.0,81.0,0.0,81.0,112.0,Arizona,7016270.0,1.275179,744.416667,Arizona,1399.694444
4,2017,134278.0,112756.0,21522.0,15458.0,1649.0,13809.0,890.0,16.0,874.0,1058.0,California,39536653.0,3.396292,1308.875000,California,2402.277778


Drop location and change `price` to `family_rent`

In [74]:
df.drop('location', axis=1, inplace=True)
df.rename(columns={'price': 'family_rent'}, inplace=True)
df.columns

Index(['year', 'total', 'individual', 'family', 'youth', 'child', 'yadult',
       'yparent', 'yparent_u18', 'yparent_18to24', 'ypchild', 'state',
       'population', 'per_capita', 'individual_rent', 'family_rent'],
      dtype='object')

Update tableau data

In [75]:
df.to_csv('datasets/tableau-data.csv')

### Create change over time dataset

Need to recognize month as a number, so strip leading `0` with `.str.lstrip("0")` and convert to int

In [34]:
rental_copy.month = rental_copy.month.str.lstrip("0")

In [35]:
rental_copy.month = rental_copy.month.astype(int)

In [36]:
rental_copy.month.head()

0    1
1    1
2    1
3    1
4    2
Name: month, dtype: int32

In [49]:
# Check that 10 month exists
rental_copy.month.unique()

array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12], dtype=int64)

Convert `year` to int in case it is needed

In [43]:
rental_copy.year = rental_copy.year.astype(int)

In [44]:
rental_copy.year.head()

0    2014
1    2015
2    2016
3    2017
4    2014
Name: year, dtype: int32

Sort data by year for ease of calculations

In [45]:
rental_copy.sort_values(by=['year', 'month'], inplace=True)

In [48]:
rental_copy.head(15)

,location,bedroom_size,month,year,price
0,Alabama,Studio,1,2014,573.0
48,Alabama,1br,1,2014,624.0
96,Alabama,2br,1,2014,758.0
144,Alabama,3br,1,2014,1013.0
192,Alabama,4br,1,2014,1178.0
240,Alaska,Studio,1,2014,841.0
288,Alaska,1br,1,2014,961.0
336,Alaska,2br,1,2014,1230.0
384,Alaska,3br,1,2014,1738.0
432,Alaska,4br,1,2014,2086.0


In [51]:
months = rental_copy.month.unique()
states = rental_copy.location.unique()
bedroom_sizes = rental_copy.bedroom_size.unique()